In [1]:
import pandas as pd
import requests
import numpy as np
import json
import datetime
import matplotlib.pyplot as plt

pd.set_option('display.max_colwidth', None)

def run_query(query):  # A simple function to use requests.post to make the API call.
    headers = {'X-API-KEY': 'BQYCaXaMZlqZrPCSQVsiJrKtxKRVcSe4'}
    request = requests.post('https://graphql.bitquery.io/', json={'query': query}, headers=headers)
    if request.status_code == 200:
        return request.json()
    else:
        print(request.reason)
        raise Exception('Query failed and return code is {}.{}'.format(request.status_code, query))

query_date = '"2021-12-16", "{}"'.format(str(datetime.date.today()))

# Convert this to JSON

VOLT_ACCOUNTS = {
    # CSP
    "SOL-P": "6Nkc8MEiz3WLz1xthYitmSuy3NGwn7782upRHo2iFmXK",
    "SOL-P-high": "JAoeEG2sfQ1xgXUNVVkJ5mCTCw4SLc6oJafDwf6jTf",
    "BTC-P": "GrB6vbG2WP7eEnbwgxUbBGRMeXYq139jo2o9oW8cNK8f",
    "SOL-P (tsUSDC)": "AQRGh6PU7LzDHvvoPNS7wVVQaCBeftw9kVDAnvuEjbs8",
    "ETH-P": "EA29Xf3HGMtYziw7UKZDUKby7gkoCbXwmiNKwc7z54Ax",
    "LUNA-P": "5kA7FPiB3t2X5s65dK1AoEu5asDjC5d7f5vaB4iY2yrj",
    "MNGO-P": "CVrRw6VtxSjokm2tKmaS5RCuoc9EFjN4wEoov6f2PST6",
#     "SOL-P-Step": ,
#     "SOL-P-Parrot": ,
#     "SOL-P-UXD": ,
    
    "SOL-C": "Hxtb6APfNtf9m8jJjh7uYp8fCTGr9aeHxBSfiPqCrV6G",
    "SOL-C-high": "DNa849drqW19uBV5X9ohpJ5brRGzq856gk3HDRqveFrA",
    "BTC-C": "DA1M8mw7GnPNKU9ReANtHPQyuVzKZtsuuSbCyc2uX2du",
    "mSOL-C": "6asST5hurmxJ8uFvh7ZRWkrMfSEzjEAJ4DNR1is3G6eH",
    "ETH-C": "FThcy5XXvab5u3jbA6NjWKdMNiCSV3oY5AAkvEvpa8wp",
    "FTT-C": "7KqHFuUksvNhrWgoacKkqyp2RwfBNdypCYgK9nxD1d6K",
    "SRM-C": "2P427N5sYcEXvZAZwqNzjXEHsBMESQoLyjNquTSmGPMb",
    "MNGO-C": "B3yakZxwomkmnCxRr8ZmQtiWgtxtVBuCREDFDdAvcCVQ",
    "scnSOL-C": "A5MpyajTy6hdsg3S2em5ukcgY1ZBhxTxEKv8BgHajv1A",
    "SBR-C": "BH7Jg3f97FyeGxsPR7FFskvfqGiaLeUnJ9Ksda53Jj8h",
    "LUNA-C": "5oV1Yf8q1oQgPYuHjepjmKFuaG2Wng9dzTqbSWhU5W2X",
    "RAY-C": "A6XsYxGj9wpqUZG81XwgQJ2zJ3efCbuWSQfnkHqUSmdM",
    "STEP-C": "FiEHDTKT6X7VFwGaUmsm1XXYr8vvkoSR5EqcY4znpefq",
    "AVAX-C": "HLfkEc6E2HEcABaWm3QZSfDvswzXjh1onF9xEzUAtWNS",
}

ACCOUNT_STRING = str(list(VOLT_ACCOUNTS.values())).replace("\'", "\"")

FEES_ADDRESS = '3KjJiWBfaw96qGhysq6Fc9FTxdPgPTNY6shM7Bwfp8EJ'

MASTER_ADDRESS = 'DxMJgeSVoe1cWo1NPExiAsmn83N3bADvkT86dSP1k7WE'

query = """
    query{
    solana(network: solana) {
    transfers(
      date: {between: ["""+ query_date +"""]}
      transferType: {is: transfer}
      any: [{senderAddress: {in: """ + ACCOUNT_STRING + """}}]
    ) {
      amount
      currency {
        symbol
        address
      }
      block {
        timestamp {
          iso8601
        }
      }
      transaction {
        signer
        signature
        innerInstructionsCount
      }
      receiver {
        address
      }
      sender {
        address
      }
    }
  }
}
"""

result = run_query(query)
# convert GraphQL json to pandas dataframe
df = pd.json_normalize(result['data']['solana']['transfers'])
df = df.rename(columns={"block.timestamp.iso8601": "date", "currency.symbol": "symbol", 
                        "transaction.signer":"signer", "currency.address":"cash",
                        "receiver.address":"receiver", "sender.address":"asset", "transaction.signature": "tx_id"})
# Does json_normalize flatten a json file for you??

df["tx_id"] = df["tx_id"].apply(lambda x: "https://solscan.io/tx/{}".format(x))
# if transaction signer & receiver is diff, drop row
df = df.query("receiver == '{}'".format(FEES_ADDRESS)).reset_index(drop=True)

def find_key(value, d): 
    for k, v in d.items():
        if value == v:
            return k

df['asset'] = df.asset.apply(lambda x: find_key(x, VOLT_ACCOUNTS))

df['symbol'] = df.apply(lambda x: "tsUSDC" if x.cash == "Cvvh8nsKZet59nsDDo3orMa3rZnPWQhpgrMCVcRDRgip" else x.symbol, axis=1)

df["FeeType"] = df["transaction.innerInstructionsCount"].apply(lambda x: "Withdrawal Fee" if x==1 else "Performance Fee")

In [2]:
df.groupby(["FeeType", "symbol"]).amount.sum()

FeeType          symbol
Performance Fee  BTC           1.222333
                 ETH           3.955708
                 FTT         493.787061
                 LUNA        465.510641
                 MNGO      22098.953302
                 RAY         231.926626
                 SBR       85893.930697
                 SOL        1645.143001
                 SRM       20159.080000
                 STEP        340.216038
                 USDC      53731.595607
                 UST        2951.350868
                 mSOL        524.010812
                 scnSOL       55.562521
                 tsUSDC     7925.858470
Withdrawal Fee   BTC           0.256253
                 ETH           1.397977
                 FTT          28.859512
                 LUNA         92.565639
                 MNGO       3499.265779
                 RAY           4.801589
                 SBR        6783.790275
                 SOL         144.859883
                 SRM         202.189150
                

# Get current coin prices porfolio calculation

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import requests
import json
import ccxt

import datetime

cftx = ccxt.ftx(
    {
        "apiKey":"mvnd3WRG56rAP-6MpxFyZ5sN2WnjEO8UsQB6u-6s",
        "secret":"m_m926E36IMlrhCv6bGkLdSdaVegbm5PggpCrFCM"
    }
)

markets = cftx.load_markets()
binance = ccxt.binance()

# 10 days
time_since = datetime.datetime.timestamp(datetime.datetime.now())*1000-10*24*60*60*1000

symbols = {
    "BTC": "BTC/USD", 
    "ETH": "ETH/USD", 
    "SOL": "SOL/USD", 
    "MSOL": "MSOL/USD", 
    "LUNA": "LUNA/USD:USD", 
    "FTT": "FTT/USD", 
    "SRM": "SRM/USD",
    "MNGO": "MNGO/USD",
    "RAY": "RAY/USD", 
    "STEP": "STEP/USD",
    "AVAX": "AVAX/USD"
}
pricedf = {}
for k, symbol in symbols.items():
    data = cftx.fetchOHLCV(symbol, '1m', since=time_since, limit=1)
    tree = pd.DataFrame(data, columns=["time", "open", "high", "low", "close", "volume"])
    data
    print(data)
    pricedf[k] = tree["close"]
    

[[1647234420000, 38512.0, 38524.0, 38509.0, 38521.0, 118380.5906]]
[[1647234420000, 2572.8, 2573.8, 2572.8, 2573.5, 18487.2315]]
[[1647234420000, 80.2625, 80.305, 80.2525, 80.2875, 10971.527]]
[[1647234420000, 83.075, 83.125, 83.0, 83.075, 0.0]]
[[1647234420000, 90.031, 90.152, 90.01, 90.042, 290635.3986]]
[[1647234420000, 40.631, 40.634, 40.625, 40.628, 304.6875]]
[[1647234420000, 1.785, 1.786, 1.785, 1.786, 0.0]]
[[1647234420000, 0.1495, 0.1495, 0.1495, 0.1495, 0.0]]
[[1647234420000, 2.3299, 2.3303, 2.3298, 2.3303, 4.6605]]
[[1647234420000, 0.141, 0.141, 0.141, 0.141, 0.0]]
[[1647234420000, 68.614, 68.668, 68.6135, 68.668, 13.7336]]


In [4]:
# Random price hardcodes b/c they aren't traded on FTX

pricedf["USDC"] =pricedf["tsUSDC"] = pricedf["UST"] = 1
pricedf["mSOL"] = pricedf["SOL"]*1.03
pricedf["scnSOL"] = pricedf["SOL"]*1.03
pricedf["SBR"] = 4000

# ALLS

In [5]:
fees = df.groupby(["FeeType", "symbol"]).amount.sum().reset_index()

tree = pd.DataFrame(pricedf).T.reset_index()

tree.columns = ["symbol", "price"]

xavier = pd.merge(fees, tree, on="symbol", how='left')

xavier["usdc_value"] = (xavier["amount"]*xavier["price"]).astype('int')

xavier.groupby(["FeeType", "symbol"])[["amount", "usdc_value"]].sum()

amount  usdc_value
FeeType         symbol                          
Performance Fee BTC         1.222333       47085
                ETH         3.955708       10180
                FTT       493.787061       20061
                LUNA      465.510641       41915
                MNGO    22098.953302        3303
                RAY       231.926626         540
                SBR     85893.930697   343575722
                SOL      1645.143001      132084
                SRM     20159.080000       36004
                STEP      340.216038          47
                USDC    53731.595607       53731
                UST      2951.350868        2951
                mSOL      524.010812       43333
                scnSOL     55.562521        4594
                tsUSDC   7925.858470        7925
Withdrawal Fee  BTC         0.256253        9871
                ETH         1.397977        3597
                FTT        28.859512        1172
                LUNA       92.565639        8334
                MNGO     3499.265779         523
                RAY         4.801589          11
                SBR      6783.790275    27135161
                SOL       144.859883       11630
                SRM       202.189150         361
                USDC    10805.127217       10805
                UST       423.862711         423
                mSOL       99.804956        8253
                scnSOL      5.244235         433
                tsUSDC   3347.567257        3347

In [6]:
xavier

,FeeType,symbol,amount,price,usdc_value
0,Performance Fee,BTC,1.222333,38521.000000,47085
1,Performance Fee,ETH,3.955708,2573.500000,10180
2,Performance Fee,FTT,493.787061,40.628000,20061
3,Performance Fee,LUNA,465.510641,90.042000,41915
4,Performance Fee,MNGO,22098.953302,0.149500,3303
5,Performance Fee,RAY,231.926626,2.330300,540
6,Performance Fee,SBR,85893.930697,4000.000000,343575722
7,Performance Fee,SOL,1645.143001,80.287500,132084
8,Performance Fee,SRM,20159.080000,1.786000,36004
9,Performance Fee,STEP,340.216038,0.141000,47


# If you want to mess with the actual withdrawals and performance fees, use this dataframe


## FeeType == "Performance Fee"


## FeeType == "Withdrawal Fee"

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 357 entries, 0 to 356
Data columns (total 10 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   amount                              357 non-null    float64
 1   symbol                              357 non-null    object 
 2   cash                                357 non-null    object 
 3   date                                357 non-null    object 
 4   signer                              357 non-null    object 
 5   tx_id                               357 non-null    object 
 6   transaction.innerInstructionsCount  357 non-null    int64  
 7   receiver                            357 non-null    object 
 8   asset                               357 non-null    object 
 9   FeeType                             357 non-null    object 
dtypes: float64(1), int64(1), object(8)
memory usage: 28.0+ KB


In [8]:
df.head(1)

,amount,symbol,cash,date,signer,tx_id,transaction.innerInstructionsCount,receiver,asset,FeeType
0,8.218337,tsUSDC,Cvvh8nsKZet59nsDDo3orMa3rZnPWQhpgrMCVcRDRgip,2022-02-06T02:35:21Z,GVs4XuqEC27ppPBsXocH6PNzLAaJhy8JjBZJdYzGbXnX,https://solscan.io/tx/4FhqyXunsNFivUtazhp6rqc5PQUADvFckV1NXrueAN1Dc5VG22brXDGVPSA5rSFb5rBA78wYY8eecfR2MJBMgaRc,4,3KjJiWBfaw96qGhysq6Fc9FTxdPgPTNY6shM7Bwfp8EJ,SOL-P (tsUSDC),Performance Fee
